<a href="https://colab.research.google.com/github/TheAmirHK/BendersDecomposition/blob/main/benders_decomposition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mip

In [31]:
from mip import Model, xsum, MINIMIZE, INTEGER, CONTINUOUS
from mip import OptimizationStatus

In [32]:
# Solving the mixed-integer model as a general form

model = Model(sense=MINIMIZE)

x1 = model.add_var(name='x1', var_type=INTEGER, lb=0)
x2 = model.add_var(name='x2', var_type=INTEGER, lb=0)
y1 = model.add_var(name='y1', var_type=CONTINUOUS, lb=0)
y2 = model.add_var(name='y2', var_type=CONTINUOUS, lb=0)
y3 = model.add_var(name='y3', var_type=CONTINUOUS, lb=0)

model.objective = 3*x1 + 22*x2 + y1 + 7*y2 + 9*y3

model += x1 + x2 + y1 - 2*y3 >= 15
model += x2 + y1 + 7*y2 >= 8

model.optimize()

if model.status == OptimizationStatus.OPTIMAL:
    print(f"Optimal x1: {x1.x}, Optimal x2: {x2.x}, Optimal y1: {y1.x}, Optimal y2: {y2.x}, Optimal y3: {y3.x}, Optimal Objective: {model.objective_value}")
else:
    print("No optimal solution found.")



Optimal x1: 0.0, Optimal x2: 0.0, Optimal y1: 15.0, Optimal y2: 0.0, Optimal y3: 0.0, Optimal Objective: 15.0


In [33]:
# Solving the mixed-integer model using Bender's decomposition method

def solve_master_problem(cuts):
    model = Model(sense=MINIMIZE)
    x1 = model.add_var(name='x1', var_type=INTEGER, lb=0)
    x2 = model.add_var(name='x2', var_type=INTEGER, lb=0)
    theta = model.add_var(name='theta', var_type=CONTINUOUS, lb=-1000)

    model.objective = 3*x1 + 22*x2 + theta

    for i, (alpha, beta) in enumerate(cuts):
        model += theta >= alpha[0] * x1 + alpha[1] * x2 + beta, f"BendersCut_{i}"

    model.optimize()
    return x1.x, x2.x, theta.x if model.status == OptimizationStatus.OPTIMAL else (None, None, None)

def solve_subproblem(x1, x2):
    model = Model(sense=MINIMIZE)
    y1 = model.add_var(name='y1', var_type=CONTINUOUS, lb=0)
    y2 = model.add_var(name='y2', var_type=CONTINUOUS, lb=0)
    y3 = model.add_var(name='y3', var_type=CONTINUOUS, lb=0)

    model.objective = y1 + 7*y2 + 9*y3

    model += x1 + x2 + y1 - 2*y3 >= 15
    model += x2 + y1 + 7*y2 >= 8

    model.optimize()

    if model.status == OptimizationStatus.OPTIMAL:
        return y1.x, y2.x, y3.x, model.objective_value
    else:
        return None, None, None, None


# benders' procedure
cuts = []
for iteration in range(10):
    x1, x2, theta = solve_master_problem(cuts)
    if x1 is None:
        break

    y1, y2, y3, obj_value = solve_subproblem(x1, x2)

    if obj_value is not None:
        alpha = [-1, -1]
        beta = obj_value
        cuts.append((alpha, beta))
    else:
        break

print(f"Optimal x1: {x1}, Optimal x2: {x2}, Optimal y1: {y1}, Optimal y2: {y2}, Optimal y3: {y3}, Optimal Objective: {obj_value}")


Optimal x1: 0.0, Optimal x2: 0.0, Optimal y1: 15.0, Optimal y2: 0.0, Optimal y3: 0.0, Optimal Objective: 15.0
